In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [2]:
# Download  2023-01 and 2023-02 yellow taxi trip data
df_2023_01 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_2023_02 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")


In [3]:
df_2023_02.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [4]:
# Get duration of each trip
df_2023_01['duration'] = df_2023_01['tpep_dropoff_datetime'] - df_2023_01['tpep_pickup_datetime']
df_2023_02['duration'] = df_2023_02['tpep_dropoff_datetime'] - df_2023_02['tpep_pickup_datetime']
# Convert duration to seconds
df_2023_01['duration'] = df_2023_01['duration'].apply(lambda x: x.total_seconds()/60)
df_2023_02['duration'] = df_2023_02['duration'].apply(lambda x: x.total_seconds()/60)

In [5]:
df_2023_01.head(5)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [6]:
df_2023_02.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00,1.683333
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00,0.233333
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00,0.233333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00,13.300000


In [7]:
print(len(df_2023_01.columns))
df_2023_01.head(5)

20


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [8]:
print(len(df_2023_02.columns))
df_2023_02

20


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.40,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.00,-1.00,-0.5,0.00,0.0,-1.0,-5.50,0.0,0.00,0.233333
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.00,1.00,0.5,0.00,0.0,1.0,5.50,0.0,0.00,0.233333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.90,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.00,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913950,2,2023-02-28 23:46:00,2023-03-01 00:05:00,NaN,4.65,NaN,None,249,140,0,20.22,0.00,0.5,4.84,0.0,1.0,29.06,NaN,NaN,19.000000
2913951,2,2023-02-28 23:26:02,2023-02-28 23:37:10,NaN,2.47,NaN,None,186,79,0,13.66,0.00,0.5,2.65,0.0,1.0,20.31,NaN,NaN,11.133333
2913952,2,2023-02-28 23:24:00,2023-02-28 23:38:00,NaN,3.49,NaN,None,158,143,0,17.64,0.00,0.5,0.00,0.0,1.0,21.64,NaN,NaN,14.000000
2913953,2,2023-02-28 23:03:00,2023-02-28 23:10:00,NaN,2.13,NaN,None,79,162,0,13.56,0.00,0.5,2.63,0.0,1.0,20.19,NaN,NaN,7.000000


In [9]:
# Get the standard deviation of the duration variable in minutes
# 2023-01
duration_std_2023_01 = df_2023_01['duration'].std()
print(duration_std_2023_01)


42.594351241920904


In [ ]:
# Get percentage of outliers in the duration variable
df_outlier_filter_2023_01 = df_2023_01[(df_2023_01.duration >= 1) & (df_2023_01.duration <= 60)]
df_outlier_filter_2023_02 = df_2023_02[(df_2023_02.duration >= 1) & (df_2023_02.duration <= 60)]
print((len(df_outlier_filter_2023_01)+len(df_outlier_filter_2023_02))/(len(df_2023_01)+len(df_2023_02)))

0.9806717283752243


In [10]:
# Get percentage of outliers in the duration variable
df_2023_01 = df_2023_01[(df_2023_01.duration >= 1) & (df_2023_01.duration <= 60)]
#df_2023_02 = df_2023_02[(df_2023_02.duration >= 1) & (df_2023_02.duration <= 60)]

In [11]:
print(df_2023_01.shape)
print(df_2023_02.shape)

(3009173, 20)
(2913955, 20)


In [12]:
# One hot encoding to pick up and drop off location ids
## Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
categorical = ['PULocationID', 'DOLocationID']
df_2023_01[categorical] = df_2023_01[categorical].astype(str)
df_2023_02[categorical] = df_2023_02[categorical].astype(str)


In [13]:
# Turn into dictionary the columns we want to one-hot encode
df_2023_01_train_dict = df_2023_01[categorical].to_dict(orient='records')
df_2023_02_val_dict = df_2023_02[categorical].to_dict(orient='records')
len(df_2023_01_train_dict)
len(df_2023_02_val_dict)


2913955

In [14]:
## Create a DictVectorizer object
dv = DictVectorizer()
## Fit the DictVectorizer object to the training data
X_train = dv.fit_transform(df_2023_01_train_dict)
X_train.shape

(3009173, 515)

In [15]:
X_val = dv.transform(df_2023_02_val_dict)

In [16]:
target = 'duration'
y_train = df_2023_01[target].values
y_val = df_2023_02[target].values

In [17]:
# Get root mean squared error
# Get the target variable
y_train = df_2023_01['duration'].values
# Fit the linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)



LinearRegression()

In [18]:
# Get the predictions
y_pred_train = lr.predict(X_train)
# Root mean squared error
training_root_mean_square = root_mean_squared_error(y_train, y_pred_train)
training_root_mean_square

7.649261932106969

In [19]:
# Get the predictions
y_pred_val = lr.predict(X_val)
val_root_mean_square = root_mean_squared_error(y_val, y_pred_val)
val_root_mean_square

42.28477577694284

# New Test

In [2]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [4]:
len(df_train), len(df_val)

(3009173, 2855951)

In [ ]:
#df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
#df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [5]:
categorical = ['PULocationID', 'DOLocationID']
#numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [6]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [7]:
lr = LinearRegression()
lr.fit(X_train, y_train)


LinearRegression()

In [8]:
# Root mean squared error Training
y_pred = lr.predict(X_train)
training_root_mean_square = root_mean_squared_error(y_train, y_pred)
training_root_mean_square

7.649261932106969

In [9]:
# Root mean squared error Validation
y_pred = lr.predict(X_val)

val_root_mean_square =root_mean_squared_error(y_val, y_pred)
val_root_mean_square

7.811818743246608

In [12]:
import pickle

In [13]:
with open('lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)